In [1]:
%cd ../
%matplotlib inline

/home/zulqarnain/featuresounddepression/htfatorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/sound_depression.db'
mask_file = None

OVERWRITE = True

In [4]:
fmriprep_dir = '/data/zulqarnain/sound_depression/fmriprep/'

In [5]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file, smooth=6)

if not existed or OVERWRITE:
    data_files = glob.glob('/data/zulqarnain/sound_depression/fmriprep/all_tasks/*_preproc.nii.gz')
    regex = re.compile("sub-([A-Za-z]+)([0-9]{1,2})_task-([A-Za-z]+)_run-([0-9]{1,2})_\S+preproc.nii.gz")
    for data_file in data_files:
        logging.info("Recording blocks in %s", data_file)
        m = regex.search(data_file)
        condition, subject, task_set, run = m.groups()
        subject = int(subject)
        run = int(run)
        if run is None:
            run = ''
        tsv_file = '/sub-%s%02d_task-%s_run-%01d_events.tsv' % (condition, subject, task_set, run)
        tsv_path = '/data/zulqarnain/sound_depression/sub-%s%02d/func/' % (condition, subject)
        tsv_file = os.path.dirname(tsv_path) + tsv_file
        with open(tsv_file, 'r') as tsv:
            tsv_reader = csv.DictReader(tsv, dialect='excel-tab')
            for row in tsv_reader:
                onset, duration, task = row.values()
                onset = round(float(onset) / 3)
                duration = round(float(duration) / 3)
                task_block = niidb.FMriActivationBlock(zscore=True)
                task_block.task = task
                task_block.filename = data_file
                task_block.subject = subject
                task_block.start_time = onset
                task_block.end_time = onset + duration
                task_block.individual_differences = {'condition': condition}
                db.upsert(task_block)
                logging.info('Stimulus %s from %d to %d', task, onset, onset + duration)

01/11/2019 18:01:31 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-mdd16_task-music_run-4_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 18:01:31 Stimulus tones from 0 to 11
01/11/2019 18:01:31 Stimulus response from 11 to 12
01/11/2019 18:01:31 Stimulus negative_music from 12 to 22
01/11/2019 18:01:31 Stimulus response from 22 to 23
01/11/2019 18:01:31 Stimulus tones from 24 to 34
01/11/2019 18:01:31 Stimulus response from 34 to 35
01/11/2019 18:01:31 Stimulus positive_music from 35 to 45
01/11/2019 18:01:31 Stimulus response from 46 to 47
01/11/2019 18:01:31 Stimulus tones from 46 to 56
01/11/2019 18:01:31 Stimulus response from 57 to 58
01/11/2019 18:01:31 Stimulus negative_music from 58 to 68
01/11/2019 18:01:31 Stimulus response from 68 to 69
01/11/2019 18:01:31 Stimulus tones from 70 to 80
01/11/2019 18:01:31 Stimulus response from 80 to 81
01/11/2019 18:01:31 Stimulus positive_music from 81 to 91
01/11/2019 18:01:31 Stimulus response from 

01/11/2019 18:01:33 Stimulus tones from 24 to 34
01/11/2019 18:01:33 Stimulus response from 34 to 35
01/11/2019 18:01:33 Stimulus negative_nonmusic from 35 to 45
01/11/2019 18:01:33 Stimulus response from 46 to 47
01/11/2019 18:01:33 Stimulus tones from 46 to 56
01/11/2019 18:01:33 Stimulus response from 57 to 58
01/11/2019 18:01:33 Stimulus positive_nonmusic from 58 to 68
01/11/2019 18:01:33 Stimulus response from 68 to 69
01/11/2019 18:01:33 Stimulus tones from 70 to 80
01/11/2019 18:01:33 Stimulus response from 80 to 81
01/11/2019 18:01:33 Stimulus negative_nonmusic from 81 to 91
01/11/2019 18:01:33 Stimulus response from 92 to 93
01/11/2019 18:01:33 Stimulus tones from 92 to 102
01/11/2019 18:01:33 Stimulus response from 103 to 104
01/11/2019 18:01:33 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-control02_task-music_run-1_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 18:01:33 Stimulus tones from 0 to 11
01/11/2019 18:01:33 Stimulus respons

01/11/2019 18:01:35 Stimulus response from 68 to 69
01/11/2019 18:01:35 Stimulus tones from 70 to 80
01/11/2019 18:01:35 Stimulus response from 80 to 81
01/11/2019 18:01:35 Stimulus positive_music from 81 to 91
01/11/2019 18:01:35 Stimulus response from 92 to 93
01/11/2019 18:01:35 Stimulus tones from 92 to 102
01/11/2019 18:01:35 Stimulus response from 103 to 104
01/11/2019 18:01:35 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-control04_task-nonmusic_run-1_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 18:01:35 Stimulus tones from 0 to 11
01/11/2019 18:01:35 Stimulus response from 11 to 12
01/11/2019 18:01:35 Stimulus negative_nonmusic from 12 to 22
01/11/2019 18:01:35 Stimulus response from 22 to 23
01/11/2019 18:01:35 Stimulus tones from 24 to 34
01/11/2019 18:01:35 Stimulus response from 34 to 35
01/11/2019 18:01:35 Stimulus positive_nonmusic from 35 to 45
01/11/2019 18:01:35 Stimulus response from 46 to 47
01/11/2019 18:01:35 Stimulus tone

01/11/2019 18:01:36 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-control08_task-nonmusic_run-4_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 18:01:36 Stimulus tones from 0 to 11
01/11/2019 18:01:36 Stimulus response from 11 to 12
01/11/2019 18:01:36 Stimulus negative_nonmusic from 12 to 22
01/11/2019 18:01:36 Stimulus response from 22 to 23
01/11/2019 18:01:37 Stimulus tones from 24 to 34
01/11/2019 18:01:37 Stimulus response from 34 to 35
01/11/2019 18:01:37 Stimulus positive_nonmusic from 35 to 45
01/11/2019 18:01:37 Stimulus response from 46 to 47
01/11/2019 18:01:37 Stimulus tones from 46 to 56
01/11/2019 18:01:37 Stimulus response from 57 to 58
01/11/2019 18:01:37 Stimulus negative_nonmusic from 58 to 68
01/11/2019 18:01:37 Stimulus response from 68 to 69
01/11/2019 18:01:37 Stimulus tones from 70 to 80
01/11/2019 18:01:37 Stimulus response from 80 to 81
01/11/2019 18:01:37 Stimulus positive_nonmusic from 81 to 91
01/11/2019 18:01:37 Stim

01/11/2019 18:01:38 Stimulus tones from 24 to 34
01/11/2019 18:01:38 Stimulus response from 34 to 35
01/11/2019 18:01:38 Stimulus positive_music from 35 to 45
01/11/2019 18:01:38 Stimulus response from 46 to 47
01/11/2019 18:01:38 Stimulus tones from 46 to 56
01/11/2019 18:01:38 Stimulus response from 57 to 58
01/11/2019 18:01:38 Stimulus negative_music from 58 to 68
01/11/2019 18:01:38 Stimulus response from 68 to 69
01/11/2019 18:01:38 Stimulus tones from 70 to 80
01/11/2019 18:01:38 Stimulus response from 80 to 81
01/11/2019 18:01:38 Stimulus positive_music from 81 to 91
01/11/2019 18:01:38 Stimulus response from 92 to 93
01/11/2019 18:01:38 Stimulus tones from 92 to 102
01/11/2019 18:01:38 Stimulus response from 103 to 104
01/11/2019 18:01:38 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-control02_task-nonmusic_run-5_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 18:01:38 Stimulus tones from 0 to 11
01/11/2019 18:01:38 Stimulus response from

...

01/11/2019 18:02:08 Stimulus response from 103 to 104
01/11/2019 18:02:08 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-control15_task-music_run-1_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 18:02:08 Stimulus tones from 0 to 11
01/11/2019 18:02:08 Stimulus response from 11 to 12
01/11/2019 18:02:08 Stimulus negative_music from 12 to 22
01/11/2019 18:02:08 Stimulus response from 22 to 23
01/11/2019 18:02:08 Stimulus tones from 24 to 34
01/11/2019 18:02:08 Stimulus response from 34 to 35
01/11/2019 18:02:08 Stimulus positive_music from 35 to 45
01/11/2019 18:02:08 Stimulus response from 46 to 47
01/11/2019 18:02:08 Stimulus tones from 46 to 56
01/11/2019 18:02:08 Stimulus response from 57 to 58
01/11/2019 18:02:08 Stimulus negative_music from 58 to 68
01/11/2019 18:02:08 Stimulus response from 68 to 69
01/11/2019 18:02:08 Stimulus tones from 70 to 80
01/11/2019 18:02:08 Stimulus response from 80 to 81
01/11/2019 18:02:08 Stimulus positive_music

01/11/2019 18:02:10 Stimulus response from 22 to 23
01/11/2019 18:02:10 Stimulus tones from 24 to 34
01/11/2019 18:02:10 Stimulus response from 34 to 35
01/11/2019 18:02:10 Stimulus negative_music from 35 to 45
01/11/2019 18:02:10 Stimulus response from 46 to 47
01/11/2019 18:02:10 Stimulus tones from 46 to 56
01/11/2019 18:02:10 Stimulus response from 57 to 58
01/11/2019 18:02:10 Stimulus positive_music from 58 to 68
01/11/2019 18:02:10 Stimulus response from 68 to 69
01/11/2019 18:02:10 Stimulus tones from 70 to 80
01/11/2019 18:02:10 Stimulus response from 80 to 81
01/11/2019 18:02:10 Stimulus negative_music from 81 to 91
01/11/2019 18:02:10 Stimulus response from 92 to 93
01/11/2019 18:02:10 Stimulus tones from 92 to 102
01/11/2019 18:02:10 Stimulus response from 103 to 104
01/11/2019 18:02:10 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-control03_task-music_run-3_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 18:02:10 Stimulus tones from 0

01/11/2019 18:02:12 Stimulus response from 57 to 58
01/11/2019 18:02:12 Stimulus negative_nonmusic from 58 to 68
01/11/2019 18:02:12 Stimulus response from 68 to 69
01/11/2019 18:02:12 Stimulus tones from 70 to 80
01/11/2019 18:02:12 Stimulus response from 80 to 81
01/11/2019 18:02:12 Stimulus positive_nonmusic from 81 to 91
01/11/2019 18:02:12 Stimulus response from 92 to 93
01/11/2019 18:02:12 Stimulus tones from 92 to 102
01/11/2019 18:02:12 Stimulus response from 103 to 104
01/11/2019 18:02:12 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-mdd01_task-music_run-3_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 18:02:12 Stimulus tones from 0 to 11
01/11/2019 18:02:12 Stimulus response from 11 to 12
01/11/2019 18:02:12 Stimulus positive_music from 12 to 22
01/11/2019 18:02:12 Stimulus response from 22 to 23
01/11/2019 18:02:12 Stimulus tones from 24 to 34
01/11/2019 18:02:12 Stimulus response from 34 to 35
01/11/2019 18:02:12 Stimulus negative_mu

01/11/2019 18:02:13 Stimulus tones from 92 to 102
01/11/2019 18:02:13 Stimulus response from 103 to 104
01/11/2019 18:02:13 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-control17_task-music_run-5_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 18:02:13 Stimulus tones from 0 to 11
01/11/2019 18:02:13 Stimulus response from 11 to 12
01/11/2019 18:02:13 Stimulus negative_music from 12 to 22
01/11/2019 18:02:13 Stimulus response from 22 to 23
01/11/2019 18:02:13 Stimulus tones from 24 to 34
01/11/2019 18:02:13 Stimulus response from 34 to 35
01/11/2019 18:02:13 Stimulus positive_music from 35 to 45
01/11/2019 18:02:13 Stimulus response from 46 to 47
01/11/2019 18:02:13 Stimulus tones from 46 to 56
01/11/2019 18:02:13 Stimulus response from 57 to 58
01/11/2019 18:02:13 Stimulus negative_music from 58 to 68
01/11/2019 18:02:13 Stimulus response from 68 to 69
01/11/2019 18:02:13 Stimulus tones from 70 to 80
01/11/2019 18:02:13 Stimulus response from 80

01/11/2019 18:02:15 Stimulus positive_music from 12 to 22
01/11/2019 18:02:15 Stimulus response from 22 to 23
01/11/2019 18:02:15 Stimulus tones from 24 to 34
01/11/2019 18:02:15 Stimulus response from 34 to 35
01/11/2019 18:02:15 Stimulus negative_music from 35 to 45
01/11/2019 18:02:15 Stimulus response from 46 to 47
01/11/2019 18:02:15 Stimulus tones from 46 to 56
01/11/2019 18:02:15 Stimulus response from 57 to 58
01/11/2019 18:02:15 Stimulus positive_music from 58 to 68
01/11/2019 18:02:15 Stimulus response from 68 to 69
01/11/2019 18:02:15 Stimulus tones from 70 to 80
01/11/2019 18:02:15 Stimulus response from 80 to 81
01/11/2019 18:02:15 Stimulus negative_music from 81 to 91
01/11/2019 18:02:15 Stimulus response from 92 to 93
01/11/2019 18:02:15 Stimulus tones from 92 to 102
01/11/2019 18:02:15 Stimulus response from 103 to 104
01/11/2019 18:02:15 Recording blocks in /data/zulqarnain/sound_depression/fmriprep/all_tasks/sub-mdd03_task-nonmusic_run-4_bold_space-MNI152NLin2009cAsym

In [6]:
logging.info('Finished building NiiDb out of Lepping 2017 dataset')

01/11/2019 18:02:16 Finished building NiiDb out of Lepping 2017 dataset
